# 명탐정 코난 매니아 판독기

## OpenAI LLM 준비 및 퀴즈 파일 지정
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`

In [ ]:
import gradio as gr
import random, json, os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 퀴즈 파일 및 출제 문항 개수 지정
QUIZ_FILE = "conan_quiz.json"
QUIZ_COUNT = 3

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)


## 퀴즈 문항 (`conan_quiz.json`) 로딩
- `def load_quiz()` :	load_quiz라는 이름의 함수를 정의
- `with open(QUIZ_FILE, "r", encoding="utf-8") as f`: 지정된 경로의 JSON 파일(QUIZ_FILE)을 UTF-8 인코딩 방식으로 읽기 모드("r")로 열고, with 구문을 사용하여 파일 사용 후 자동으로 닫히도록 처리
- `all_q = json.load(f)` : JSON 파일 내용을 파싱하여 파이썬 객체(리스트 형태, 퀴즈 문제들)로 변환.
- `return random.sample(all_q, QUIZ_COUNT)` :전체 문제 중 QUIZ_COUNT 개수만큼 무작위로 샘플링하여 반환(중복 없이 선택)

In [ ]:
# 퀴즈 로딩 함수
def load_quiz():
    with open(QUIZ_FILE, "r", encoding="utf-8") as f:
        all_q = json.load(f)
    return random.sample(all_q, QUIZ_COUNT)

## 문제 출력 구성
퀴즈 상태 정보(`state`)를 기반으로, 현재 문제를 텍스트로 출력해주는 역할로 선다형(`multiple_choice`) 문제에 대해 선택지를 함께 출력하도록 구성
- `def get_question(state)` : 퀴즈 상태 객체(`state`)를 받아 현재 문제를 문자열로 구성해 반환하는 함수
- `idx = state["quiz_index"]` : 현재 퀴즈 진행 인덱스(첫 번째 문제는 0)
- `q = state["questions"][idx]` : 문제 리스트 중 현재 인덱스에 해당하는 문제 데이터 (각 문제는 딕셔너리 구조)
- `text = f"문제 {idx+1}: {q['question']}"` : 문제 텍스트를 구성 (idx+1로 번호를 1부터 시작하도록 표시) - 예) 문제 1: 코난의 본명은?
- `if q["type"] == "multiple_choice"` : 현재 문제가 **선다형(`multiple_choice`)**인지 확인
- `choices = [f"{i+1}. {c}" for i, c in enumerate(q["choices"])]` : 선다형 선택지를 1. 보기 형식으로 나열 - 예) ["1. 쿠도 신이치", "2. 하이바라 아이", ...]
- `text += "\n" + "\n".join(choices)` : 문제 텍스트에 줄바꿈 후 선택지들을 한 줄씩 추가
- `return text` : 최종적으로 구성된 문제 문자열을 반환

In [ ]:
# 문제 출력 - 선다형 구성
def get_question(state):
    idx = state["quiz_index"]
    q = state["questions"][idx]
    text = f"문제 {idx+1}: {q['question']}"
    if q["type"] == "multiple_choice":
        choices = [f"{i+1}. {c}" for i, c in enumerate(q["choices"])]
        text += "\n" + "\n".join(choices)
    return text

## 사용자 응답 구성
사용자의 답변을 현재 상태(`state`)에 저장하고, 퀴즈 인덱스를 다음 문제로 넘기는 역할
- 사용자 입력 저장 : 현재 문제에 대한 사용자 응답을 저장
- 응답 전처리 : 선다형일 경우 숫자를 선택지 텍스트로 변환
- 다음 문제로 진행: `quiz_index`를 1 증가시켜 다음 문제로 이동 준비

In [ ]:
# 사용자 답변을 상태에 저장
def update_state(state, user_input):
    idx = state["quiz_index"]
    q = state["questions"][idx]
    processed = user_input.strip()               # 사용자 입력 앞뒤 공백 제거

    if q["type"] == "multiple_choice":
        try:
            sel = int(processed) - 1             # 사용자 입력을 선다형 인덱스로 변환 (1 작은 수)
            if 0 <= sel < len(q["choices"]):     # 인덱스 유효 범위 확인
                processed = q["choices"][sel]
        except:
            pass                                 # 입력 상태 유지

    state["user_answers"].append(                # 사용자 응답 결과를 기록하는 리스트에 새로운 항목을 추가
        {
            "question_text": q["question"],      # 퀴즈 문항 질문
            "user_response": processed,          # 사용자 답변
            "is_correct": False,                 # 정답 여부는 아직 채점 전이므로 일단 False로 저장
            "correct_answer": str(q["answer"]),  # 정답은 문자열로 변환해서 저장
        }
    )
    state["quiz_index"] += 1                     # 다음 문제로 넘어가기 위해 인덱스를 1 증가
    return state

## LLM으로 보낼 채점 프롬프트
LLM에 퀴즈 채점을 위한 프롬프트로 사용자 답변의 정답 여부 및 피드백을 요청할 때 사용
- 사용자 응답(`user_answers`)과 문제(`questions`) 정보를 기반으로, 채점 요청 프롬프트를 LLM에게 전달하기 위해 생성
- 전체 퀴즈를 하나의 긴 문자열 프롬프트로 반환 (문제 + 정답 + 사용자 답변)
- 선다형이면 선택지까지 제공함으로써 LLM이 문맥 기반으로 정확히 판단할 수 있도록 구성

In [ ]:
# 채점 프롬프트 생성
def build_grading_prompt(state):
    parts = [
        "당신은 퀴즈 채점관입니다. 사용자 답변을 정답 여부로 판단하고 각 문제에 피드백을 제공해주세요.",
        "마지막에는 '총점: X/Y' 형식으로 출력해주세요.",
    ]
    for i, (q, a) in enumerate(zip(state["questions"], state["user_answers"])):
        parts.append(f"\n문제 {i+1}: {q['question']}")
        if q["type"] == "multiple_choice":
            parts.append(f"선택지: {', '.join(q['choices'])}")
        parts.append(f"정답: {q['answer']}")
        parts.append(f"사용자 답변: {a['user_response']}")
    return "\n".join(parts)

## LCEL 채점 파이프라인 구성
LangChain Expression Language(LCEL)를 사용하여 채점용 LLM 체인을 구성
- `ChatPromptTemplate.from_messages(...)` : 역할 기반 메시지 형식의 프롬프트를 정의
- `("user", "{grading_input}")` : 퀴즈 질문과 사용자의 응답을 채점할 내용 (`build_grading_prompt()` 함수로 생성된 프롬프트)
- `StrOutputParser()` : LLM의 응답을 문자열로 변환

In [ ]:
# LCEL 채점 체인
grade_chain = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "채점관으로서 정답 판단 및 피드백을 제공해주세요."),
            ("user", "{grading_input}"),
        ]
    )
    | llm
    | StrOutputParser()
)

### 챗봇 메인 함수 내 - 퀴즈 시작 요청 처리
사용자가 "퀴즈" 또는 "퀴즈 시작"이라고 입력했을 때, 퀴즈를 초기화하고 첫 번째 문제를 메시지에 추가
- `quiz_state["questions"] = load_quiz()` : JSON에서 퀴즈 문제를 불러와 `quiz_state` 딕셔너리에 저장
- `messages.append([user_input, qtext])` : 사용자 입력과 챗봇 응답을 쌍으로 저장하여 채팅 히스토리를 구성

In [ ]:
# 퀴즈 시작 요청 처리
def handle_quiz_start(user_input, quiz_state, messages):
    quiz_state["questions"] = load_quiz()
    quiz_state["quiz_index"] = 0
    quiz_state["user_answers"] = []
    qtext = get_question(quiz_state)
    messages.append([user_input, qtext])
    return quiz_state, messages

### 챗봇 메인 함수 내 - 퀴즈가 이미 끝난 경우
퀴즈가 이미 끝났는데도 사용자가 계속 입력할 경우, 안내 메시지를 보냄

In [ ]:
# 퀴즈가 이미 끝난 경우
def handle_quiz_already_done(user_input, messages):
    messages.append(
        [
            user_input,
            "퀴즈가 이미 종료되었습니다. 다시 시작하려면 '퀴즈 시작'이라고 입력하세요.",
        ]
    )
    return messages

### 챗봇 메인 함수 내 - 사용자 답변 처리
사용자의 답변을 저장하고, 다음 문제를 보여주거나, 모든 문제를 마쳤을 경우 채점을 수행
- `if quiz_state["quiz_index"] < len(quiz_state["questions"])` : 현재 퀴즈가 아직 진행 중인지 확인
- `prompt = build_grading_prompt(quiz_state)` : 사용자 답변들과 정답을 LLM에게 넘기기 위한 프롬프트(문자열)를 생성

In [ ]:
# 사용자 답변 처리
def handle_user_answer(user_input, quiz_state, messages):
    quiz_state = update_state(quiz_state, user_input)

    if quiz_state["quiz_index"] < len(quiz_state["questions"]):
        qtext = get_question(quiz_state)
        messages.append([user_input, qtext])
    else:
        prompt = build_grading_prompt(quiz_state)
        result = grade_chain.invoke({"grading_input": prompt})
        messages.append([user_input, result])

    return quiz_state, messages

## 메인 챗봇 처리 함수
챗봇의 메인 제어 함수. 사용자의 입력에 따라 퀴즈를 시작할지, 계속할지, 종료 안내할지를 판단
- `user_input_lower = user_input.strip().lower()` : 사용자 입력에서 공백을 제거하고 소문자로 변환

In [ ]:
# 메인 챗봇 처리 함수
def chat_fn(user_input, state):
    user_input_lower = user_input.strip().lower()
    messages = state["chat_history"]
    quiz_state = state["quiz_state"]

    if quiz_state["questions"] == []:
        if user_input_lower in ["퀴즈", "퀴즈 시작"]:
            quiz_state, messages = handle_quiz_start(user_input, quiz_state, messages)
        else:
            messages.append(
                [
                    user_input,
                    "'퀴즈' 또는 '퀴즈 시작'이라고 입력하면 퀴즈를 시작합니다.",
                ]
            )
    elif quiz_state["quiz_index"] >= len(quiz_state["questions"]):
        messages = handle_quiz_already_done(user_input, messages)
    else:
        quiz_state, messages = handle_user_answer(user_input, quiz_state, messages)

    state["quiz_state"] = quiz_state
    state["chat_history"] = messages
    return messages, state

## 초기 상태 정의
퀴즈를 새로 시작하거나 첫 실행 시 사용할 수 있는 초기 상태 딕셔너리를 생성해 반환

In [ ]:
# 상태 초기화
def init_state():
    return {
        "quiz_state": {"quiz_index": 0, "questions": [], "user_answers": []},
        "chat_history": [],
    }

## Gradio UI 정의
Gradio의 Blocks UI 구성 방식을 사용해 명탐정 코난 퀴즈 챗봇 인터페이스를 구성
- `with gr.Blocks() as demo `: Gradio의 레이아웃 기반 UI 블록을 시작 - 앱 전체 인스턴스를 `demo`로 선언
- `gr.Markdown(...)` : 화면 상단에 표시할 설명 문구 (Markdown 문법 사용)
- `gr.Chatbot(...)` : 사용자와 AI 간의 대화를 보여주는 대화창 컴포넌트
- `gr.Textbox(...)` : 사용자가 입력할 수 있는 텍스트 입력창 (placeholder로 입력 힌트를 표시)
- `gr.State(init_state())` : `init_state()`를 호출하여 내부 상태를 저장 - 퀴즈 진행 정보 등을 여기에 보관

※ 입력 처리 상세 설명

1. `txt.submit(chat_fn, inputs=[txt, state], outputs=[chatbot, state])`
    - `txt.submit(...)` : 사용자가 텍스트박스에 입력하고 Enter를 누르면 실행
    - `chat_fn` : 메인 처리 함수 - 사용자의 입력을 받아 챗봇 응답과 상태를 계산 또는 
    - `inputs=[txt, state]` : 텍스트 입력값과 이전 상태(state)를 chat_fn에 전달
    - `outputs=[chatbot, state]` : 함수 결과로 나온 대화 내용과 상태를 챗봇 창과 내부 상태에 각각 반영

2. `txt.submit(lambda: "", None, txt)`
    - `lambda: ""` : 아무 동작 없이 빈 문자열을 반환하는 함수
    - `None` : 입력값은 없음
    - `txt` : 출력 대상은 텍스트박스 자신 - 입력 후 자동으로 입력창을 초기화

In [ ]:
# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### 🕵️ 명탐정 코난 매니아 판별기")
    chatbot = gr.Chatbot(label="명탐정 코난 퀴즈 챗봇", height=400)
    txt = gr.Textbox(placeholder="'퀴즈 시작'을 입력해보세요!", show_label=False)
    state = gr.State(init_state())

    txt.submit(chat_fn, inputs=[txt, state], outputs=[chatbot, state])
    txt.submit(lambda: "", None, txt)

demo.launch()

-----
** End of Documents **